# Twinstake Staking API Cookbook

Requirements:
- Python>=3.11
- requests
- boto3
- fireblocks >=6.0.0 ([SDK](https://github.com/fireblocks/py-sdk))


SDKs:
- Using **Fireblocks** to manage vaults & tokens
- Using **Twinstake API** to build staking transactions and broadcast them to the network

In [3]:
import requests
import boto3
import time
import json

from fireblocks.client import Fireblocks
from fireblocks.client_configuration import ClientConfiguration
from fireblocks.base_path import BasePath
from fireblocks.models.vault_account import VaultAccount
from fireblocks.models.transaction_request import TransactionRequest, TransactionOperation
from fireblocks.models.transaction_response import TransactionResponse
from fireblocks.models.transfer_peer_path_type import TransferPeerPathType
from fireblocks.models.source_transfer_peer_path import SourceTransferPeerPath

## Fireblocks SDK Setup

- Provided by Fireblocks
- Built by the client on its own
- Used to get the public key, address on a blockchain and sign transactions

TODO:
- Update using your fireblocks secret key location, API key & Firbelocks API Path

In [4]:
YOUR_SECRET_KEY_LOCATION = './...'
YOUR_API_KEY = ''
FIREBLOCKS_BASE_PATH = BasePath.Sandbox

with open(YOUR_SECRET_KEY_LOCATION, 'r') as file:
    secret_key_value = file.read()

configuration = ClientConfiguration(
        api_key=YOUR_API_KEY,
        secret_key=secret_key_value,
        base_path=FIREBLOCKS_BASE_PATH
)

In [5]:
class FireblocksFlow:
    """Fireblocks class to manage vaults & tokens"""
    def __init__(self, configuration: ClientConfiguration):
        self.client: Fireblocks = Fireblocks(configuration)
        self.vaults: VaultAccount = self._get_vault_account()

    def _get_vault_account(self) -> VaultAccount:
        api_response = self.client.vaults.get_paged_vault_accounts().result()
        return api_response.data.accounts
    
    @property
    def vault_ids(self) -> list[str]:
        return [vault.id for vault in self.vaults]

    def get_public_key(self, vault_id: str, asset_id: str) -> str:
        change = 0
        address_index = 0
        api_response = self.client.vaults.get_public_key_info_for_address(
            vault_id, asset_id, change, address_index, True
        ).result()
        return api_response.data.public_key

    def get_addresses(self, vault_id: str, asset_id: str) -> str:
        try:
            api_response = self.client.vaults.get_vault_account_asset_addresses_paginated(vault_id, asset_id).result()
            return [address.address for address in api_response.data.addresses]
        except Exception as e:
            print(e)
            print(f"Error getting asset addresse {asset_id} for vault {vault_id}")
            return None

    def get_transaction_info(self, tx_id: str) -> dict:
        return self.client.transactions.get_transaction(tx_id).result()

    def sign_message(self, message_hash: str, vault_id: str, asset_id: str) -> str:
        extra = {
            "rawMessageData": {
                "messages": [
                    {
                        "content": message_hash,
                    }
                ]
            }
        }
        
        tx_request = TransactionRequest(
            operation=TransactionOperation.RAW,
            asset_id=asset_id,
            source=SourceTransferPeerPath(
                type=TransferPeerPathType.VAULT_ACCOUNT,
                id=vault_id
            ),
            extraParameters=extra
        )

        return self.client.transactions.create_transaction(
            transaction_request=tx_request
        ).result()

## Twinstake API Setup

Sending API requests to Twinstake
We are providing documentation for the API in our [**documentation page**](https://docs.twinstake.io/)

Fill in your API credentials and the Twinstake API URL you want to use (mainnet or testnet)

- Testnet 
    - Client ID: `5auqhut2p16as4erujcq0kbu9k`
    - API URL: `https://dev.api.twinstake.io/v2/`

- Mainnet 
    - Client ID: `2t0af84tabi78j5et1b7pbjiqa`
    - API URL: `https://api.twinstake.io/v2/`

In [6]:
TWINSTAKE_API_URL = "https://dev.api.twinstake.io/v2/" # https://dev.api.twinstake.io/v2/
TWINSTAKE_CLIENT_ID = ""
TWINSTAKE_USERNAME = ""
TWINSTAKE_PASSWORD = ""

def twinstake_api_post(token, endpoint, payload={}):
    resp = requests.post(
        f"{TWINSTAKE_API_URL}{endpoint}",
        headers={"Authorization": "Bearer " + token},
        json=payload,
    )
    if resp.status_code == 200:
        return resp.json()
    return resp

def twinstake_api_get(token, endpoint):
    resp = requests.get(
        f"{TWINSTAKE_API_URL}{endpoint}",
        headers={"Authorization": "Bearer " + token},
    )
    if resp.status_code == 200:
        return resp.json()
    print(f"Error: {resp.status_code}!")
    return resp

def get_cognito_auth():
    client = boto3.client("cognito-idp", "eu-west-1")
    resp = client.initiate_auth(
        AuthFlow="USER_PASSWORD_AUTH",
        AuthParameters={"USERNAME": TWINSTAKE_USERNAME, "PASSWORD": TWINSTAKE_PASSWORD},
        ClientId=TWINSTAKE_CLIENT_ID,
    )
    return resp["AuthenticationResult"]["IdToken"]

## Setup

- Choose the asset you want to stake. Available assets can be retrieved from the Fireblocks API
    - `fb_flow.client.blockchains_assets.get_supported_assets().result().data`
- Input the Twinstake Validator address you want to stake to

Testnets validators:
- `Solana: vgcDar2pryHvMgPkKaZfh8pQy4BJxv7SpwUG7zinWjG`
- `Cosmos: cosmosvaloper1apac0g5s88pfjvlgjqjnr0kdpkpmhx8pne4608`
- `Celestia: celestiavaloper1jt9w26mpxxjsk63mvd4m2ynj0af09cslh5d096`
- `Near: kiln.pool.f863973.m0`
- `Polygon: 0x00bd5b5d50c60a0702EDF6536b1e4897ea2d7f9f`

In [ ]:
ASSET = "NEAR_TEST"#"ETH_TEST5"#"NEAR_TEST" #"OM", "ATOM", ...
VAULT_ID = "1" # Your Fireblocks Vault ID containing the assets
VALIDATOR = "kiln.pool.f863973.m0"#"0x00bd5b5d50c60a0702EDF6536b1e4897ea2d7f9f" # Twinstake Validator Address
ENTITY_FILTER = "" # Your Twinstake Entity Filter to use for the given asset

fb_flow = FireblocksFlow(configuration)
address = fb_flow.get_addresses(VAULT_ID, ASSET)[0]
public_key = fb_flow.get_public_key(VAULT_ID, ASSET)
print(f"📝 Fireblocks Address: {address}")

# Solana Staking with Twinstake and Fireblocks

ASSET = `"SOL_TEST"` for devnet, `"SOL"` for mainnet

Cookbook:
- [Creating a Solana Stake Account](#1-creating-a-solana-stake-account)
- [Delegating the Stake](#2-delegating-the-stake)
- [Deactivating the Stake](#3-deactivating-the-stake)
- [Withdrawing the Stake](#4-withdrawing-the-stake)


### 1. Creating a Solana Stake Account

- Creating a stake account with the amount of SOL to stake
- Know what will be the stake account address

In [ ]:
SOL_AMOUNT = 0.01 # in $SOL

create_stake_account = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/solana/create-stake-account", 
    payload={
        "amount": SOL_AMOUNT,
        "delegator_address": address,
        "withdraw_authority_address": address,
        "stake_authority_address": address,
        "entity_filter": ENTITY_FILTER
    }
)
print(f"🧪 ...New Stake Account: {create_stake_account['stake_account_address']}")

In [ ]:
# Fireblocks Raw Signing
signed_request = fb_flow.sign_message(create_stake_account["transaction"]["hash"], VAULT_ID, ASSET)
while True:
    tx_info: TransactionResponse = fb_flow.get_transaction_info(signed_request.data.id).data
    if tx_info.status == "COMPLETED":
        print("✍🏻 Transaction Signed")
        break
    else:
        time.sleep(1)
        continue

In [ ]:
# Broadcasting the transaction to the network
signature = tx_info.signed_messages[0].to_dict()["signature"]['fullSig']
broadcast_create_stake_account = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/solana/broadcast",
    payload={
        "unsigned_transaction": create_stake_account['transaction']['hex'],
        "signatures": [signature],
    }
)
print(f"🎉 Broadcasted Transaction: {broadcast_create_stake_account['transaction_hash']}")

In [ ]:
# Checking the status of the transaction
tx_status = twinstake_api_get(
    token=get_cognito_auth(), 
    endpoint=f"staking/solana/transactions/{broadcast_create_stake_account['transaction_hash']}", 
)
print("🔎 ✨ Successfully found transaction on the network")
print(tx_status)

### 2 - Delegate the stake to the Validator

- Using the stake account address previously created, delegate the stake to the validator

In [62]:
delegate_sol = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/solana/delegate", 
    payload={
        "delegator_address": address,
        "stake_account_address": create_stake_account['stake_account_address'],
        "validator_address": VALIDATOR,
        "entity_filter": ENTITY_FILTER
    }
)

In [ ]:
# Fireblocks Raw Signing
signed_request = fb_flow.sign_message(delegate_sol["transaction"]["hash"], VAULT_ID, ASSET)
while True:
    tx_info: TransactionResponse = fb_flow.get_transaction_info(signed_request.data.id).data
    if tx_info.status == "COMPLETED":
        print("✍🏻 Transaction Signed")
        break
    else:
        time.sleep(1)
        continue

In [ ]:
# Broadcasting the transaction to the network
signature = tx_info.signed_messages[0].to_dict()["signature"]['fullSig']
broadcast_delegate_sol = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/solana/broadcast", 
    payload={
        "unsigned_transaction": delegate_sol['transaction']['hex'],
        "signatures": [signature],
    }
)
print(f"🎉 Broadcasted Transaction: {broadcast_delegate_sol['transaction_hash']}")

In [ ]:
tx_status = twinstake_api_get(
    token=get_cognito_auth(), 
    endpoint=f"staking/solana/transactions/{broadcast_delegate_sol['transaction_hash']}", 
)
print("🔎 ✨ Successfully found transaction on the network")
print(tx_status)

### 3 - Deactivate the stake

- Deactivate the stake from the validator using the stake account address you have

In [66]:
deactivate_sol = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/solana/deactivate", 
    payload={
        "delegator_address": address,
        "stake_account_address": create_stake_account['stake_account_address'],
        "validator_address": VALIDATOR,
        "entity_filter": ENTITY_FILTER
    }
)
signed_request = fb_flow.sign_message(deactivate_sol["transaction"]["hash"], VAULT_ID, ASSET)

In [ ]:
while True:
    tx_info: TransactionResponse = fb_flow.get_transaction_info(signed_request.data.id).data
    if tx_info.status == "COMPLETED":
        print("✍🏻 Transaction Signed")
        break
    else:
        time.sleep(1)
        continue

In [ ]:
signature = tx_info.signed_messages[0].to_dict()["signature"]['fullSig']
broadcast_deactivate_sol = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/solana/broadcast", 
    payload={
        "unsigned_transaction": deactivate_sol['transaction']['hex'],
        "signatures": [signature],
    }
)
print(f"🎉 Broadcasted Transaction: {broadcast_deactivate_sol['transaction_hash']}")

In [ ]:
tx_status = twinstake_api_get(
    token=get_cognito_auth(), 
    endpoint=f"staking/solana/transactions/{broadcast_deactivate_sol['transaction_hash']}", 
)
print("🔎 ✨ Successfully Deactivated Stake")
print(tx_status)

### 4 - Withdraw the stake

- Withdraw the stake from the validator using the stake account address you have

In [71]:
withdraw_sol = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/solana/withdraw", 
    payload={
        "delegator_address": address,
        "stake_account_address": create_stake_account['stake_account_address'],
        "entity_filter": ENTITY_FILTER
    }
)
signed_request = fb_flow.sign_message(withdraw_sol["transaction"]["hash"], VAULT_ID, ASSET)

In [ ]:
while True:
    tx_info: TransactionResponse = fb_flow.get_transaction_info(signed_request.data.id).data
    if tx_info.status == "COMPLETED":
        print("✍🏻 Transaction Signed")
        break
    else:
        time.sleep(1)
        continue

In [ ]:
signature = tx_info.signed_messages[0].to_dict()["signature"]['fullSig']
broadcast_withdraw_sol = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/solana/broadcast", 
    payload={
        "unsigned_transaction": withdraw_sol['transaction']['hex'],
        "signatures": [signature],
    }
)
print(f"🎉 Broadcasted Transaction: {broadcast_withdraw_sol['transaction_hash']}")

In [ ]:
tx_status = twinstake_api_get(
    token=get_cognito_auth(), 
    endpoint=f"staking/solana/transactions/{broadcast_withdraw_sol['transaction_hash']}", 
)
print("🔎 ✨ Successfully Withdrawn Stake")
print(tx_status)

# Tendermint Staking with Twinstake and Fireblocks

ASSET = `"CELESTIA_TEST"` for devnet, `"CELESTIA"` for mainnet
for other tendermint assets, you can check the available assets ticker with the fireblocks SDK

Cookbook:
- [Delegate Tendermint asset to a validator](#1-delegate-tendermint-asset-to-a-validator)
- [Claim Tendermint rewards](#2-claiming-tendermint-rewards)
- [Undelegate Tendermint asset](#3-undelegate-tendermint-asset)


### 1. Delegate Tendermint asset to a validator

- Delegate the Tendermint asset to a validator

In [85]:
AMOUNT = 0.1 # in $TOKEN

delegate_tia = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/celestia/delegate", 
    payload={
        "amount": AMOUNT,
        "delegator_address": address,
        "delegator_public_key": public_key,
        "validator_address": VALIDATOR,
        "entity_filter": ENTITY_FILTER
    }
)

In [ ]:
signed_request = fb_flow.sign_message(delegate_tia["transaction"]["hash"], VAULT_ID, ASSET)
while True:
    tx_info: TransactionResponse = fb_flow.get_transaction_info(signed_request.data.id).data
    if tx_info.status == "COMPLETED":
        print("✍🏻 Transaction Signed")
        break
    else:
        time.sleep(1)
        continue

In [ ]:
# Broadcasting the transaction to the network
signature = tx_info.signed_messages[0].to_dict()["signature"]['fullSig']
broadcast_delegate_tia = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/celestia/broadcast", 
    payload={
        "unsigned_transaction": delegate_tia['transaction']['hex'],
        "signatures": [signature],
    }
)
print(f"🎉 Broadcasted Transaction: {broadcast_delegate_tia['transaction_hash']}")

In [ ]:
tx_status = twinstake_api_get(
    token=get_cognito_auth(), 
    endpoint=f"staking/celestia/transactions/{broadcast_delegate_tia['transaction_hash']}", 
)
print("🔎 ✨ Successfully found transaction on the network")
print(tx_status)

### 2. Claiming Tendermint rewards

In [150]:
claim_tia = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/celestia/rewards/claim", 
    payload={
        "delegator_address": address,
        "delegator_public_key": public_key,
        "validator_address": VALIDATOR,
        "entity_filter": ENTITY_FILTER
    }
)

In [ ]:
signed_request = fb_flow.sign_message(claim_tia["transaction"]["hash"], VAULT_ID, ASSET)
while True:
    tx_info: TransactionResponse = fb_flow.get_transaction_info(signed_request.data.id).data
    if tx_info.status == "COMPLETED":
        print("✍🏻 Transaction Signed")
        break
    else:
        time.sleep(1)
        continue

In [ ]:
# Broadcasting the transaction to the network
signature = tx_info.signed_messages[0].to_dict()["signature"]['fullSig']
broadcast_claim_tia = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/celestia/broadcast", 
    payload={
        "unsigned_transaction": claim_tia['transaction']['hex'],
        "signatures": [signature],
    }
)
print(f"🎉 Broadcasted Transaction: {broadcast_claim_tia['transaction_hash']}")

In [ ]:
tx_status = twinstake_api_get(
    token=get_cognito_auth(), 
    endpoint=f"staking/celestia/transactions/{broadcast_claim_tia['transaction_hash']}", 
)
print("🔎 ✨ Successfully found transaction on the network")
print(tx_status)

### 3 - Undelegate Tendermint asset

- Undelegate the Tendermint asset from the validator
- Specify `"amount"` information to undelegate a specific amount 
- Specify `"unstake_all"` to undelegate all the stake

In [14]:
undelegate_tia = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/celestia/undelegate", 
    payload={
        "unstake_all": True,
        "delegator_address": address,
        "delegator_public_key": public_key,
        "validator_address": VALIDATOR,
        "entity_filter": ENTITY_FILTER
    }
)

In [ ]:
signed_request = fb_flow.sign_message(undelegate_tia["transaction"]["hash"], VAULT_ID, ASSET)
while True:
    tx_info: TransactionResponse = fb_flow.get_transaction_info(signed_request.data.id).data
    if tx_info.status == "COMPLETED":
        print("✍🏻 Transaction Signed")
        break
    else:
        time.sleep(1)
        continue

In [ ]:
# Broadcasting the transaction to the network
signature = tx_info.signed_messages[0].to_dict()["signature"]['fullSig']
broadcast_undelegate_tia = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/celestia/broadcast", 
    payload={
        "unsigned_transaction": undelegate_tia['transaction']['hex'],
        "signatures": [signature],
    }
)
print(f"🎉 Broadcasted Transaction: {broadcast_undelegate_tia['transaction_hash']}")

In [ ]:
tx_status = twinstake_api_get(
    token=get_cognito_auth(), 
    endpoint=f"staking/celestia/transactions/{broadcast_undelegate_tia['transaction_hash']}", 
)
print("🔎 ✨ Successfully found transaction on the network")
print(tx_status)

# Near Staking with Twinstake and Fireblocks

ASSET = `"NEAR_TEST"` for devnet, `"NEAR"` for mainnet

Cookbook:
- [Delegate Near tokens](#1-delegate-near-tokens)
- [Undelegate Near rewards](#2-undelegate-near-rewards)
- [Withdraw Near tokens](#3-withdraw-near-tokens)

### 1 - Delegate Near tokens

In [25]:
AMOUNT = 0.1 # in $TOKEN

delegate_near = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/near/delegate", 
    payload={
        "amount": AMOUNT,
        "delegator_address": address,
        "delegator_public_key": public_key,
        "validator_address": VALIDATOR,
        "entity_filter": ENTITY_FILTER
    }
)

In [ ]:
signed_request = fb_flow.sign_message(delegate_near["transaction"]["hash"], VAULT_ID, ASSET)
while True:
    tx_info: TransactionResponse = fb_flow.get_transaction_info(signed_request.data.id).data
    if tx_info.status == "COMPLETED":
        print("✍🏻 Transaction Signed")
        break
    else:
        time.sleep(1)
        continue

In [28]:
signature = tx_info.signed_messages[0].to_dict()["signature"]['fullSig']

In [ ]:
# Broadcasting the transaction to the network
#signature = tx_info.signed_messages[0].to_dict()["signature"]['fullSig']
broadcast_delegate_near = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/near/broadcast", 
    payload={
        "unsigned_transaction": delegate_near['transaction']['raw'],
        "signatures": [signature],
    }
)
print(f"🎉 Broadcasted Transaction: {broadcast_delegate_near['transaction_hash']}")

In [ ]:
tx_status = twinstake_api_get(
    token=get_cognito_auth(), 
    endpoint=f"staking/near/transactions/{broadcast_delegate_near['transaction_hash']}?sender_account_id={address}", 
)
print("🔎 ✨ Successfully found transaction on the network")
print(tx_status)

### 2 - Undelegate Near Rewards

input "amount" to undelegate a specific amount

Note: If you want to undelegate all the rewards, specify `"unstake_all": True`

In [107]:
undelegate_near = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/near/undelegate", 
    payload={
        "delegator_address": address,
        "delegator_public_key": public_key,
        "validator_address": VALIDATOR,
        "entity_filter": ENTITY_FILTER
    }
)

In [ ]:
signed_request = fb_flow.sign_message(undelegate_near["transaction"]["hash"], VAULT_ID, ASSET)
while True:
    tx_info: TransactionResponse = fb_flow.get_transaction_info(signed_request.data.id).data
    if tx_info.status == "COMPLETED":
        print("✍🏻 Transaction Signed")
        break
    else:
        time.sleep(1)
        continue

In [ ]:
# Broadcasting the transaction to the network
signature = tx_info.signed_messages[0].to_dict()["signature"]['fullSig']
broadcast_undelegate_near = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/near/broadcast", 
    payload={
        "unsigned_transaction": undelegate_near['transaction']['raw'],
        "signatures": [signature],
    }
)
print(f"🎉 Broadcasted Transaction: {broadcast_undelegate_near['transaction_hash']}")

In [ ]:
tx_status = twinstake_api_get(
    token=get_cognito_auth(), 
    endpoint=f"staking/near/transactions/{broadcast_undelegate_near['transaction_hash']}?sender_account_id={address}", 
)
print("🔎 ✨ Successfully found transaction on the network")
print(tx_status)

### 3 - Withdraw Near Tokens

In [113]:
withdraw_near = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/near/withdraw", 
    payload={
        "delegator_address": address,
        "delegator_public_key": public_key,
        "validator_address": VALIDATOR,
        "entity_filter": ENTITY_FILTER
    }
)

In [ ]:
signed_request = fb_flow.sign_message(withdraw_near["transaction"]["hash"], VAULT_ID, ASSET)
while True:
    tx_info: TransactionResponse = fb_flow.get_transaction_info(signed_request.data.id).data
    if tx_info.status == "COMPLETED":
        print("✍🏻 Transaction Signed")
        break
    else:
        time.sleep(1)
        continue

In [ ]:
# Broadcasting the transaction to the network
signature = tx_info.signed_messages[0].to_dict()["signature"]['fullSig']
broadcast_withdraw_near = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/near/broadcast", 
    payload={
        "unsigned_transaction": withdraw_near['transaction']['raw'],
        "signatures": [signature],
    }
)
print(f"🎉 Broadcasted Transaction: {broadcast_withdraw_near['transaction_hash']}")

In [ ]:
tx_status = twinstake_api_get(
    token=get_cognito_auth(), 
    endpoint=f"staking/near/transactions/{broadcast_withdraw_near['transaction_hash']}?sender_account_id={address}", 
)
print("🔎 ✨ Successfully found transaction on the network")
print(tx_status)

# Polygon Staking with Twinstake and Fireblocks

ASSET = `"ETH_TEST5"` for testnet, `"ETH"` for mainnet

Cookbook:
- [Set Allowance for Polygon Tokens and delegate](#1-set-allowance-for-polygon-tokens-and-delegate)
- [Delegate Polygon tokens](#2-claim-polygon-rewards)
- [Undelegate Polygon rewards](#2-unstake-polygon-rewards)
- [Withdraw Polygon tokens](#3-claim-unstaked-polygon-tokens)


### 1 - Set Allowance for Polygon Tokens and delegate

In [8]:
AMOUNT = 0.1 # in $POL
allowance_pol = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/polygon/allowance", 
    payload={
        "amount": AMOUNT,
        "delegator_address": address,
        "validator_address": VALIDATOR,
        "entity_filter": ENTITY_FILTER
    }
)

In [ ]:
signed_request = fb_flow.sign_message(allowance_pol["transaction"]["hash"], VAULT_ID, ASSET)
while True:
    tx_info: TransactionResponse = fb_flow.get_transaction_info(signed_request.data.id).data
    if tx_info.status == "COMPLETED":
        print("✍🏻 Transaction Signed")
        break
    else:
        time.sleep(1)
        continue

In [ ]:
# Broadcasting the transaction to the network
signature = tx_info.signed_messages[0].to_dict()["signature"]['fullSig']
broadcast_allowance_pol = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/polygon/broadcast", 
    payload={
        "unsigned_transaction": allowance_pol["transaction"]["hex"],
        "signatures": [signature],
        "delegator_address": address
    }
)
print(f"🎉 Broadcasted Transaction: {broadcast_allowance_pol['transaction_hash']}")

In [ ]:
tx_status = twinstake_api_get(
    token=get_cognito_auth(), 
    endpoint=f"staking/polygon/transactions/{broadcast_allowance_pol['transaction_hash']}", 
)
print("🔎 ✨ Successfully found transaction on the network")
print(tx_status)

In [24]:
delegate_pol = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/polygon/delegate", 
    payload={
        "amount": AMOUNT,
        "delegator_address": address,
        "validator_address": VALIDATOR,
        "entity_filter": ENTITY_FILTER
    }
)

In [ ]:
signed_request = fb_flow.sign_message(delegate_pol["transaction"]["hash"], VAULT_ID, ASSET)
while True:
    tx_info: TransactionResponse = fb_flow.get_transaction_info(signed_request.data.id).data
    if tx_info.status == "COMPLETED":
        print("✍🏻 Transaction Signed")
        break
    else:
        time.sleep(1)
        continue

In [ ]:
# Broadcasting the transaction to the network
signature = tx_info.signed_messages[0].to_dict()["signature"]['fullSig']
broadcast_delegate_pol = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/polygon/broadcast", 
    payload={
        "unsigned_transaction": delegate_pol["transaction"]["hex"],
        "signatures": [signature],
        "delegator_address": address
    }
)
print(f"🎉 Broadcasted Transaction: {broadcast_delegate_pol['transaction_hash']}")

In [ ]:
tx_status = twinstake_api_get(
    token=get_cognito_auth(), 
    endpoint=f"staking/polygon/transactions/{broadcast_delegate_pol['transaction_hash']}", 
)
print("🔎 ✨ Successfully found transaction on the network")
print(tx_status)

### 2 - Claiming Polygon rewards

In [34]:
claim_rewards_pol = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/polygon/claim_rewards", 
    payload={
        "delegator_address": address,
        "validator_address": VALIDATOR,
        "entity_filter": ENTITY_FILTER
    }
)

In [ ]:
signed_request = fb_flow.sign_message(claim_rewards_pol["transaction"]["hash"], VAULT_ID, ASSET)
while True:
    tx_info: TransactionResponse = fb_flow.get_transaction_info(signed_request.data.id).data
    if tx_info.status == "COMPLETED":
        print("✍🏻 Transaction Signed")
        break
    else:
        time.sleep(1)
        continue

In [ ]:
# Broadcasting the transaction to the network
signature = tx_info.signed_messages[0].to_dict()["signature"]['fullSig']
broadcast_claim_rewards_pol = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/polygon/broadcast", 
    payload={
        "unsigned_transaction": claim_rewards_pol["transaction"]["hex"],
        "signatures": [signature],
        "delegator_address": address
    }
)
print(f"🎉 Broadcasted Transaction: {broadcast_claim_rewards_pol['transaction_hash']}")

In [ ]:
tx_status = twinstake_api_get(
    token=get_cognito_auth(), 
    endpoint=f"staking/polygon/transactions/{broadcast_claim_rewards_pol['transaction_hash']}", 
)
print("🔎 ✨ Successfully found transaction on the network")
print(tx_status)

### 3 - Unstake Polygon tokens

In [9]:
claim_unbond_pol = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/polygon/unbond", 
    payload={
        "amount": AMOUNT,
        "delegator_address": address,
        "validator_address": VALIDATOR,
        "entity_filter": ENTITY_FILTER
    }
)

In [ ]:
signed_request = fb_flow.sign_message(claim_unbond_pol["transaction"]["hash"], VAULT_ID, ASSET)
while True:
    tx_info: TransactionResponse = fb_flow.get_transaction_info(signed_request.data.id).data
    if tx_info.status == "COMPLETED":
        print("✍🏻 Transaction Signed")
        break
    else:
        time.sleep(1)
        continue

In [ ]:
# Broadcasting the transaction to the network
signature = tx_info.signed_messages[0].to_dict()["signature"]['fullSig']
broadcast_unbond_pol = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/polygon/broadcast", 
    payload={
        "unsigned_transaction": claim_unbond_pol["transaction"]["hex"],
        "signatures": [signature],
        "delegator_address": address
    }
)
print(f"🎉 Broadcasted Transaction: {broadcast_unbond_pol['transaction_hash']}")

In [ ]:
tx_status = twinstake_api_get(
    token=get_cognito_auth(), 
    endpoint=f"staking/polygon/transactions/{broadcast_unbond_pol['transaction_hash']}", 
)
print("🔎 ✨ Successfully found transaction on the network")
print(tx_status)

### 4 - Claim Unstaked Polygon tokens

In [13]:
claim_unbond_claim_pol = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/polygon/unbond_claim", 
    payload={
        "amount": AMOUNT,
        "delegator_address": address,
        "validator_address": VALIDATOR,
        "entity_filter": ENTITY_FILTER
    }
)

In [ ]:
signed_request = fb_flow.sign_message(claim_unbond_claim_pol["transaction"]["hash"], VAULT_ID, ASSET)
while True:
    tx_info: TransactionResponse = fb_flow.get_transaction_info(signed_request.data.id).data
    if tx_info.status == "COMPLETED":
        print("✍🏻 Transaction Signed")
        break
    else:
        time.sleep(1)
        continue

In [ ]:
# Broadcasting the transaction to the network
signature = tx_info.signed_messages[0].to_dict()["signature"]['fullSig']
broadcast_unbond_claim_pol = twinstake_api_post(
    token=get_cognito_auth(), 
    endpoint="staking/polygon/broadcast", 
    payload={
        "unsigned_transaction": claim_unbond_claim_pol["transaction"]["hex"],
        "signatures": [signature],
        "delegator_address": address
    }
)
print(f"🎉 Broadcasted Transaction: {broadcast_unbond_claim_pol['transaction_hash']}")

In [ ]:
tx_status = twinstake_api_get(
    token=get_cognito_auth(), 
    endpoint=f"staking/polygon/transactions/{broadcast_unbond_claim_pol['transaction_hash']}", 
)
print("🔎 ✨ Successfully found transaction on the network")
print(tx_status)

In [ ]:
tx_status.json()